In [1]:
import numpy as np
import keras2onnx
import onnxruntime
from keras.models import Sequential, load_model, Model
from keras.datasets import mnist
from metrics import *
import tensorflow.compat.v1 as tf
from glob import glob
import os
import facenet
from keras.preprocessing import image

Using TensorFlow backend.
Can't import tf2onnx module, so the conversion on a model with any custom/lambda layer will fail!
W0427 22:06:43.707475 139985405609792 wrapper.py:21] Can't import tf2onnx module, so the conversion on a model with any custom/lambda layer will fail!
W0427 22:06:43.938368 139985405609792 deprecation.py:323] From /home/aswin/anaconda3/envs/enscalo_test/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from PIL import Image
import cv2

In [ ]:
(X, y), (X_test, y_test) = mnist.load_data()
X = X[:, :, :, np.newaxis].astype('float32') / 255
X_test = X_test[:, :, :, np.newaxis].astype('float32') / 255

In [ ]:
def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(160, 160))
    return np.expand_dims(np.asarray(img),axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [ ]:
classes, nrof_classes = facenet.get_dataset("vggface2_train_2/train")
path_exp = os.path.expanduser("vggface2_train_2/train")
image_paths = [glob(os.path.join(path_exp, classes[i], "*.jpg")) for i in range(nrof_classes)]
split = int(round(nrof_classes*(1-0.99)))
X_test = paths_to_tensor([image_paths[0][0],image_paths[0][1]])

In [19]:
img = Image.open("vggface2_test_2/test/n001898/0002_01.jpg").convert('L')
face_cascade = cv2.CascadeClassifier('/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml')
img = np.asarray(img)
faces_multi = face_cascade.detectMultiScale(img, 1.3, 5)
faces = [face.astype(np.int64).tolist() for face in faces_multi]
yes_face = faces[0]
img = cv2.resize(img[yes_face[1]:yes_face[1]+yes_face[3],yes_face[0]:yes_face[0]+yes_face[2]],(160,160))

In [20]:
arcface_model = load_model('models/mnist_vgg8_arcface_5d/model_sm.hdf5', custom_objects={'ArcFace': ArcFace})
arcface_model = Model(inputs=arcface_model.input[0], outputs=arcface_model.layers[-3].output)
arcface_features = arcface_model.predict(img.reshape(1,160,160,1), verbose=1)
arcface_features /= np.linalg.norm(arcface_features, axis=1, keepdims=True)

[(None, 5), (None, 1000)]
1/1 [==============================] - 0s 315ms/step


In [6]:
arcface_features

array([[-0.176519  , -0.105028  ,  0.95071614, -0.21184155, -0.09524728]],
      dtype=float32)

In [9]:
arcface_features

array([[-0.17389667, -0.10519306,  0.93706113, -0.26787877, -0.09408321]],
      dtype=float32)

In [12]:
arcface_features

array([[-0.17534307, -0.09559851,  0.9498372 , -0.2157349 , -0.1066931 ]],
      dtype=float32)

In [15]:
arcface_features

array([[-0.17858912, -0.09505837,  0.9690808 , -0.09122588, -0.10784248]],
      dtype=float32)

In [21]:
arcface_features

array([[-0.17978127, -0.1065986 ,  0.9706176 , -0.04294081, -0.11123333]],
      dtype=float32)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig2 = plt.figure()
ax2 = Axes3D(fig2)
for c in range(len(np.unique(y_test))):
    ax2.plot(arcface_features[y_test==c, 0], arcface_features[y_test==c, 1], arcface_features[y_test==c, 2], '.', alpha=0.1)
plt.title('ArcFace')

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
train_generator = datagen.flow_from_directory(
        'data/train',
        target_size=(1580, 862),
        batch_size=32)
validation_generator = datagen.flow_from_directory(
        'data/validation',
        target_size=(1580, 862),
        batch_size=32,
        class_mode='binary')


In [ ]:
import imagehash, cv2
from PIL import Image

In [ ]:
imagehash.average_hash(
    Image.open("vggface2_test/test/n000029/0130_01.jpg")
)

In [ ]:
from mtcnn import MTCNN
import cv2
import numpy as np
import matplotlib.pyplot as plt